In [1]:
!pip install protobuf

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!ls goose

B1.png	B3.png	B5.png	C2.png	C4.png	C6.png
B2.png	B4.png	C1.png	C3.png	C5.png


In [1]:
import cv2


In [2]:
import cv2
import numpy as np

def stitch_images(images):
    # Initialize OpenCV's Stitcher class
    stitcher = cv2.Stitcher_create()
    
    # Stitch the images together
    status, stitched = stitcher.stitch(images)
    
    # Check if the stitching was successful
    if status == cv2.Stitcher_OK:
        return stitched
    else:
        print('Image stitching failed.')
        return None


In [3]:
from glob import glob
files = glob("goose/*.png")
files_cv2 = [cv2.imread(i) for i in files]

In [4]:
# # Load your images - ensure they are in the correct order for stitching
# image1 = cv2.imread('path/to/your/image1.png')
# image2 = cv2.imread('path/to/your/image2.png')
# images = [image1, image2]

# Stitch the images
stitched_image = stitch_images(files_cv2)

# If stitching was successful, save or display the stitched image
if stitched_image is not None:
    cv2.imwrite('stitched_image.png', stitched_image)
    # # Opetionally display the result
    # cv2.imshow('Stitched Image', stitched_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [6]:
files

['goose/B1.png',
 'goose/C1.png',
 'goose/B2.png',
 'goose/B3.png',
 'goose/B4.png',
 'goose/C6.png',
 'goose/B5.png',
 'goose/C2.png',
 'goose/C3.png',
 'goose/C5.png',
 'goose/C4.png']

In [7]:
import cv2
import numpy as np

def align_images(base_img, img_to_align):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Find the keypoints and descriptors with SIFT
    keypoints_base, descriptors_base = sift.detectAndCompute(base_img, None)
    keypoints_to_align, descriptors_to_align = sift.detectAndCompute(img_to_align, None)

    # Create BFMatcher object and match descriptors
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors_base, descriptors_to_align, k=2)

    # Apply ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Extract location of good matches
    points_base = np.zeros((len(good_matches), 2), dtype=np.float32)
    points_to_align = np.zeros_like(points_base)

    for i, match in enumerate(good_matches):
        points_base[i, :] = keypoints_base[match.queryIdx].pt
        points_to_align[i, :] = keypoints_to_align[match.trainIdx].pt

    # Find homography
    H, _ = cv2.findHomography(points_to_align, points_base, cv2.RANSAC)

    # Use homography to warp images
    height, width, channels = base_img.shape
    aligned_img = cv2.warpPerspective(img_to_align, H, (width, height))
    
    return aligned_img

In [8]:
images = files_cv2

In [9]:
base_img = images[0]

# Align each image to the base image
aligned_images = [base_img]  # The first image is already aligned with itself
for img in images[1:]:
    aligned_img = align_images(base_img, img)
    aligned_images.append(aligned_img)

# Save or process aligned images
for i, aligned_img in enumerate(aligned_images):
    cv2.imwrite(f'aligned_image_{i}.png', aligned_img)

In [1]:
from PIL import Image

def reduce_png_file_size(image_path, output_path, resize_factor=None):
    # Load the image
    img = Image.open(image_path)

    # Optionally resize the image
    if resize_factor is not None and 0 < resize_factor < 1:
        new_width = int(img.width * resize_factor)
        new_height = int(img.height * resize_factor)
        img = img.resize((new_width, new_height), Image.ANTIALIAS)

    # Save the image with optimization
    img.save(output_path, optimize=True, quality=85)

# Example usage
image_path = 'resized_image_image.png'
output_path = 'goose_smaller_b1_b2_sift_ransac.png'
reduce_png_file_size(image_path, output_path, resize_factor=0.5)  # Resize to 50% of the original dimensions


In [1]:
!ls goose

B1.png	B3.png	B5.png	C2.png	C4.png	C6.png
B2.png	B4.png	C1.png	C3.png	C5.png


In [ ]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('goose/B1.png')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Initialize the SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints in the image
keypoints = sift.detect(gray, None)

# Draw the keypoints on the image
keypoint_image = np.zeros_like(image)
keypoint_image = cv2.drawKeypoints(image, keypoints, keypoint_image, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display the image with keypoints
cv2.imshow('SIFT Keypoints', keypoint_image)
cv2.waitKey(0)
cv2.destroyAllWindows()